In [10]:
from PIL import Image
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
import os

In [ ]:
# Convert all pages of the PDF to a list of images
images = convert_from_path('rules/all_cards.pdf')

# Don't want the backs of the pages
backs = set([2, 4, 6, 8, 11, 12, 14, 16, 18, 21, 22, 24])

# Define your grid - adjust based on your specific grid layout
num_rows = 3
num_cols = 3

for i, page in enumerate(images):
    if i + 1 in backs:
        continue
    os.mkdir('images/page_{}'.format(i))
    width, height = page.size
    cell_width = width // num_cols
    cell_height = height // num_rows
    images = []
    # Save each cell as an image
    for r in range(num_rows):
        for c in range(num_cols):
            left = c * cell_width
            upper = r * cell_height
            right = (c + 1) * cell_width
            lower = (r + 1) * cell_height
            cell = page.crop((left, upper, right, lower))
            # images.append(cell)
            cell.save('images/page_{}/{}_{}.png'.format(i, r, c), 'PNG')
    fig, axes = plt.subplots(3, 3, figsize=(8, 8))

    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i])
            ax.axis('off')  # Hide axes

    plt.show()
    

In [4]:
from datasets import load_dataset
dataset = load_dataset('imagefolder', data_dir='images')

Resolving data files: 100%|██████████| 108/108 [00:00<00:00, 95727.99it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 108 examples [00:00, 870.85 examples/s] 


In [ ]:
! accelerate config

In [6]:

! export MODEL_NAME="runwayml/stable-diffusion-v1-5"
! export DATA_DIR="./data"


In [ ]:
! accelerate launch textual_inversion.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$DATA_DIR \
  --learnable_property="object" \
  --placeholder_token="<dixit>" --initializer_token="artwork" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 --scale_lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir="textual_inversion_dixit" \
  --validation_prompt="A <dixit> scene in the woods" \
  --num_validation_images=4 \
  --validation_steps=10